# BDMLtools分箱算法介绍并与scorecardpy对比  <font size=3 >by 曾珂</font>

In [1]:
import scorecardpy as sc
import BDMLtools as bm
import pandas as pd

+ load germancredit data

In [2]:
dt=sc.germancredit().copy()
dt['creditability']=dt['creditability'].map({'good':0,'bad':1})

+ 将germancredit的有序category类型转换为有序数值类型
+ bm.dtypeAllocator作用在于分配指定列类型至number和object两种类型并限制数据精度
    + bm库的分箱及评分卡制作限于这两种类型

In [3]:
da=bm.dtypeAllocator().fit(dt)
dt=da.transform(dt)

In [4]:
X=dt.drop('creditability',axis=1)
y=dt['creditability']

## 分箱对比

### 分箱-tree

总结:

+ BDMLtools的tree分箱的底层使用numpy并按照scorecardpy分箱逻辑进行了重写
+ tree最优分箱中两者的分箱结果大体一致,在一些细节上有不同
+ BDMLtools的tree分箱运行效率更佳
+ scorecardpy使用mutilprocessing完成列处理的并行,BDMLtools则使用joblib完成列处理的并行

+ scorecardpy

In [5]:
%%time
bins_sc = sc.woebin(dt, y = "creditability",method="tree",bin_num_limit=5,count_distr_limit=0.05,no_cores=1)

[INFO] creating woe binning ...
CPU times: user 3.3 s, sys: 27.9 ms, total: 3.33 s
Wall time: 3.33 s


+ BDMLtools

In [6]:
%%time
bins_bm=bm.binSelector(method='tree',bin_num_limit=5,distr_limit=0.05,iv_limit=0,n_jobs=1).fit(X,y).bins

CPU times: user 564 ms, sys: 3.81 ms, total: 568 ms
Wall time: 566 ms


+ 对比结果

In [7]:
bins_sc['credit.amount']

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,credit.amount,"[-inf,1400.0)",267,0.267,185,82,0.307116,0.033661,0.000305,0.18122,1400.0,False
1,credit.amount,"[1400.0,1800.0)",105,0.105,87,18,0.171429,-0.728239,0.046815,0.18122,1800.0,False
2,credit.amount,"[1800.0,4000.0)",382,0.382,287,95,0.248691,-0.258307,0.024109,0.18122,4000.0,False
3,credit.amount,"[4000.0,9200.0)",196,0.196,120,76,0.387755,0.390539,0.031987,0.18122,9200.0,False
4,credit.amount,"[9200.0,inf)",50,0.050,21,29,0.580000,1.170071,0.078005,0.18122,inf,False


In [8]:
bins_bm['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.18122,0.009048,0.148571,1400.0
"[1400.0, 1800.0)",credit.amount,105,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.18122,0.055238,0.148571,1800.0
"[1800.0, 4000.0)",credit.amount,382,0.382,287.0,95.0,0.248691,-0.258307,0.024109,0.18122,0.148571,0.148571,4000.0
"[4000.0, 9200.0)",credit.amount,196,0.196,120.0,76.0,0.387755,0.390539,0.031987,0.18122,0.066667,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.18122,0.000000,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.18122,0.000000,0.148571,special
missing,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.18122,0.000000,0.148571,missing


+ iv对比

In [9]:
pd.concat(
    [pd.concat(bins_sc).groupby('variable')['bin_iv'].sum().rename('iv_sc'),
     pd.concat(bins_bm).groupby('variable')['bin_iv'].sum().rename('iv_bm')],axis=1
)

,iv_sc,iv_bm
variable,,
age.in.years,0.130499,0.130499
credit.amount,0.181220,0.181220
credit.history,0.291830,0.291830
duration.in.month,0.282618,0.282618
foreign.worker,0.000000,0.000000
housing,0.083293,0.083293
installment.rate.in.percentage.of.disposable.income,0.025569,0.026322
job,0.008095,0.008095
number.of.existing.credits.at.this.bank,0.010084,0.010084


### 分箱chi2merge

+ BDMLtools的chi2分箱的底层使用numpy并按照scorecardpy的分箱逻辑进行了重写
+ chi2分箱中两者的分箱结果大体一致,部分不一致在于BDMLtools修复了scorecardpy中卡方分箱的breaks与bin对应不一致问题
+ BDMLtools的chi2分箱运行效率更佳
+ scorecardpy使用mutilprocessing完成列处理的并行,BDMLtools则使用joblib完成列处理的并行

In [10]:
%%time
bins_sc_chi = sc.woebin(dt, y = "creditability",method="chimerge",bin_num_limit=5,count_distr_limit=0.05,no_cores=1)

[INFO] creating woe binning ...
CPU times: user 4.86 s, sys: 32.8 ms, total: 4.89 s
Wall time: 4.91 s


In [11]:
%%time
bins_bm_chi=bm.binSelector(method='chi2',bin_num_limit=5,distr_limit=0.05,iv_limit=0,n_jobs=1).fit(X,y).bins

CPU times: user 1.08 s, sys: 6.62 ms, total: 1.09 s
Wall time: 1.09 s


+ 结果对比

In [12]:
bins_sc_chi['age.in.years']

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,age.in.years,"[-inf,26.0)",190,0.190,110,80,0.421053,0.528844,0.057921,0.123935,26.0,False
1,age.in.years,"[26.0,35.0)",358,0.358,246,112,0.312849,0.060465,0.001324,0.123935,35.0,False
2,age.in.years,"[35.0,37.0)",79,0.079,67,12,0.151899,-0.872488,0.048610,0.123935,37.0,False
3,age.in.years,"[37.0,inf)",373,0.373,277,96,0.257373,-0.212371,0.016080,0.123935,inf,False


In [13]:
bins_bm_chi['age.in.years']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 26.0)",age.in.years,190.0,0.190,110.0,80.0,0.421053,0.528844,0.057921,0.123935,0.109524,0.131429,26.0
"[26.0, 35.0)",age.in.years,358.0,0.358,246.0,112.0,0.312849,0.060465,0.001324,0.123935,0.131429,0.131429,35.0
"[35.0, 37.0)",age.in.years,79.0,0.079,67.0,12.0,0.151899,-0.872488,0.048610,0.123935,0.075714,0.131429,37.0
"[37.0, inf)",age.in.years,373.0,0.373,277.0,96.0,0.257373,-0.212371,0.016080,0.123935,0.000000,0.131429,inf
special,age.in.years,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.123935,0.000000,0.131429,special
missing,age.in.years,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.123935,0.000000,0.131429,missing


In [14]:
pd.concat(
    [pd.concat(bins_sc_chi).groupby('variable')['bin_iv'].sum().rename('iv_sc'),
     pd.concat(bins_bm_chi).groupby('variable')['bin_iv'].sum().rename('iv_bm')],axis=1
)

,iv_sc,iv_bm
variable,,
age.in.years,0.123935,0.123935
credit.amount,0.171431,0.114711
credit.history,0.291829,0.246868
duration.in.month,0.282618,0.282618
foreign.worker,0.000000,0.000000
housing,0.083293,0.038878
installment.rate.in.percentage.of.disposable.income,0.023859,0.019769
job,0.000000,0.000000
number.of.existing.credits.at.this.bank,0.000000,0.000000


## BDMLtools的补充

### BDMLtools对scorecardpy分箱算法的补充1:更多的分箱算法

bm.binSelector的method:
+ ‘freq’:数值等频分箱，分类特征按其类别分箱
+ ‘freq-kmeans’:基于Kmeans，对freq-cut结果进行自动调整，以将badrate近似的箱进行合并
+ 'pretty':使用Pretty Breakpoints获取数值特征分箱点
    + pretty分箱点更加美观，适合报告、绘图
    + 详见R的pretty函数           
+ 'tree':决策树,分裂iv/ks增益最高的切分点形成新分箱直到达到终止条件
+ 'chi2':卡方,先进行预分箱,再合并低于卡方值(交叉表卡方检验的差异不显著)的分箱

#### 等频

In [15]:
bin_freq=bm.binSelector(method='freq',bin_num_limit=5,iv_limit=0,n_jobs=1).fit(X,y).bins

In [16]:
bin_freq['age.in.years']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 26.0)",age.in.years,190,0.190,110.0,80.0,0.421053,0.528844,0.057921,0.087812,0.109524,0.122381,26.0
"[26.0, 30.0)",age.in.years,181,0.181,124.0,57.0,0.314917,0.070068,0.000901,0.087812,0.122381,0.122381,30.0
"[30.0, 36.0)",age.in.years,217,0.217,156.0,61.0,0.281106,-0.091684,0.001790,0.087812,0.102857,0.122381,36.0
"[36.0, 45.0)",age.in.years,211,0.211,159.0,52.0,0.246445,-0.270363,0.014548,0.087812,0.049048,0.122381,45.0
"[45.0, inf)",age.in.years,201,0.201,151.0,50.0,0.248756,-0.257959,0.012652,0.087812,0.000000,0.122381,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.087812,0.000000,0.122381,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.087812,0.000000,0.122381,missing


#### 等频-kmeans分箱算法

In [17]:
bin_kmeans=bm.binSelector(method='freq-kmeans',max_bin=5,bin_num_limit=4,iv_limit=0,n_jobs=1).fit(X,y).bins

In [18]:
bin_kmeans['age.in.years']
#注意等频5箱中[36.0, 45.0)和[45.0, Inf)的badprob非常接近,此时算法自动合并了这两箱

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 26.0)",age.in.years,190.0,0.190,110.0,80.0,0.421053,0.528844,0.057921,0.087797,0.109524,0.122381,26.0
"[26.0, 30.0)",age.in.years,181.0,0.181,124.0,57.0,0.314917,0.070068,0.000901,0.087797,0.122381,0.122381,30.0
"[30.0, 36.0)",age.in.years,217.0,0.217,156.0,61.0,0.281106,-0.091684,0.001790,0.087797,0.102857,0.122381,36.0
"[36.0, inf)",age.in.years,412.0,0.412,310.0,102.0,0.247573,-0.264302,0.027185,0.087797,0.000000,0.122381,inf
special,age.in.years,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.087797,0.000000,0.122381,special
missing,age.in.years,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.087797,0.000000,0.122381,missing


#### pretty分箱

+ 对比freq分箱,pretty分箱会损失信息量但能够获取到美观的breaks
+ scorecardpy在卡方、决策树分箱的预分箱中都采用的是pretty分箱而非等频/等距等分箱,BDMLtools与scorecardpy一致

In [19]:
bin_pretty=bm.binSelector(method='pretty',bin_num_limit=5,distr_limit=0.05,iv_limit=0,n_jobs=1).fit(X,y).bins

In [20]:
bin_pretty['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1800.0)",credit.amount,372.0,0.372,272.0,100.0,0.268817,-0.153334,0.008470,0.086088,0.055238,0.108095,1800.0
"[1800.0, 2600.0)",credit.amount,181.0,0.181,129.0,52.0,0.287293,-0.061271,0.000671,0.086088,0.066190,0.108095,2600.0
"[2600.0, 3400.0)",credit.amount,121.0,0.121,92.0,29.0,0.239669,-0.307195,0.010679,0.086088,0.100952,0.108095,3400.0
"[3400.0, 5800.0)",credit.amount,165.0,0.165,117.0,48.0,0.290909,-0.043675,0.000312,0.086088,0.108095,0.108095,5800.0
"[5800.0, inf)",credit.amount,161.0,0.161,90.0,71.0,0.440994,0.610168,0.065956,0.086088,0.000000,0.108095,inf
special,credit.amount,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.086088,0.000000,0.108095,special
missing,credit.amount,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.086088,0.000000,0.108095,missing


In [21]:
bin_freq['credit.amount'] 

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1258.0)",credit.amount,195,0.195,134.0,61.0,0.312821,0.060332,0.000718,0.092071,0.011905,0.119048,1258.0
"[1258.0, 1872.0)",credit.amount,195,0.195,148.0,47.0,0.241026,-0.299767,0.016416,0.092071,0.042857,0.119048,1872.0
"[1872.0, 2762.0)",credit.amount,195,0.195,144.0,51.0,0.261538,-0.190690,0.006810,0.092071,0.078571,0.119048,2762.0
"[2762.0, 4454.0)",credit.amount,195,0.195,145.0,50.0,0.256410,-0.217413,0.008800,0.092071,0.119048,0.119048,4454.0
"[4454.0, inf)",credit.amount,220,0.220,129.0,91.0,0.413636,0.498345,0.059327,0.092071,0.000000,0.119048,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.092071,0.000000,0.119048,special
missing,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.092071,0.000000,0.119048,missing


### BDMLtools对scorecardpy分箱算法的补充2:强制单调

bm.binSelector可以强制任意分箱算法实现数值特征的单调分箱,以tree为例
+ 参数coerce_monotonic=True即可实现单调分箱
+ 细节请查看binSelector的帮助文档

In [22]:
bin_monotonic=bm.binSelector(method='tree',bin_num_limit=5,distr_limit=0.05,
               iv_limit=0,n_jobs=1,coerce_monotonic=True).fit(X,y).bins

+ age.in.years

In [23]:
bin_monotonic['age.in.years']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 26.0)",age.in.years,190,0.190,110.0,80.0,0.421053,0.528844,0.057921,0.100182,0.109524,0.131429,26.0
"[26.0, 30.0)",age.in.years,181,0.181,124.0,57.0,0.314917,0.070068,0.000901,0.100182,0.122381,0.131429,30.0
"[30.0, 35.0)",age.in.years,177,0.177,122.0,55.0,0.310734,0.050610,0.000458,0.100182,0.131429,0.131429,35.0
"[35.0, inf)",age.in.years,452,0.452,344.0,108.0,0.238938,-0.311213,0.040902,0.100182,0.000000,0.131429,inf
special,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.100182,0.000000,0.131429,special
missing,age.in.years,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.100182,0.000000,0.131429,missing


+ credit.amount

In [24]:
bin_monotonic['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 4000.0)",credit.amount,754,0.754,559.0,195.0,0.258621,-0.205852,0.030584,0.141682,0.148571,0.148571,4000.0
"[4000.0, 6800.0)",credit.amount,133,0.133,83.0,50.0,0.375940,0.340480,0.016375,0.141682,0.100476,0.148571,6800.0
"[6800.0, 9200.0)",credit.amount,63,0.063,37.0,26.0,0.412698,0.494476,0.016718,0.141682,0.066667,0.148571,9200.0
"[9200.0, inf)",credit.amount,50,0.050,21.0,29.0,0.580000,1.170071,0.078005,0.141682,0.000000,0.148571,inf
special,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.141682,0.000000,0.148571,special
missing,credit.amount,0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.141682,0.000000,0.148571,missing


In [25]:
#所有变量的效果
##字符类特征(purpose)是否单调意义不大,可忽略其单调性结果
bin_monotonic=bm.binSelector(method='tree',bin_num_limit=5,distr_limit=0.05,
               iv_limit=0,n_jobs=1,coerce_monotonic=True).fit(X,y).bins

for key in bin_monotonic:
    
    vtab=bin_monotonic[key]
    
    badprob=vtab.loc[~vtab.index.isin(['missing','special'])]['badprob']
    
    if badprob.is_monotonic_decreasing or badprob.is_monotonic_increasing:
        
        print('monotonic trend shows in {}'.format(key))
    
    else:
        
        print('no monotonic trend shows in {}'.format(key))

monotonic trend shows in age.in.years
monotonic trend shows in credit.amount
monotonic trend shows in credit.history
monotonic trend shows in duration.in.month
monotonic trend shows in foreign.worker
monotonic trend shows in housing
monotonic trend shows in installment.rate.in.percentage.of.disposable.income
monotonic trend shows in job
monotonic trend shows in number.of.existing.credits.at.this.bank
monotonic trend shows in number.of.people.being.liable.to.provide.maintenance.for
monotonic trend shows in other.debtors.or.guarantors
monotonic trend shows in other.installment.plans
monotonic trend shows in personal.status.and.sex
monotonic trend shows in present.employment.since
monotonic trend shows in present.residence.since
monotonic trend shows in property
monotonic trend shows in purpose
monotonic trend shows in savings.account.and.bonds
monotonic trend shows in status.of.existing.checking.account
monotonic trend shows in telephone


### BDMLtools对scorecardpy分箱算法的补充3:样本权重

若数据是经过抽样获取的，一般希望通过样本加权建模以利于还原其违约概率、ln(好坏比)等指标。
+ 加入样本权重后各个分箱的坏样本率会产生一定变化
+ 加入样本权重后各个分箱的iv,ks也会产生变化(若只对好坏样本加权且breaks一致则无变化)
+ 在分箱算法中样本权重的影响如下:
    + 每一箱的样本数会产生一定变化,进而影响箱样本量。
        + 例如卡方分箱的交叉表频数统计会产生变化进而影响卡方值的计算结果
        + 影响通过每箱最小样本比例控制的任何分箱算法的结果
    + 坏样本率会产生一定变化,进而影响单调分箱结果
    + iv,ks等会产生一定变化,进而影响tree最优分箱结果
    
总之加入样本权重的分箱算法会更加精准的给出breaks

#### 例:加入样本权重后的卡方分箱结果

In [26]:
sample_weight=y.map({0:10,1:1}) #假定数据经过抽样后好样本权重为10,坏样本权重为1

In [27]:
#无加权(卡方分箱)
bin_nw=bm.binSelector(method='chi2',bin_num_limit=5,distr_limit=0.05,n_jobs=1,sample_weight=None).fit(X,y).bins

In [28]:
#有加权(卡方分箱)
bin_w=bm.binSelector(method='chi2',bin_num_limit=5,distr_limit=0.05,n_jobs=1,sample_weight=sample_weight).fit(X,y).bins

In [29]:
bin_nw['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,267.0,0.267,185.0,82.0,0.307116,0.033661,0.000305,0.114711,0.009048,0.117619,1400.0
"[1400.0, 1800.0)",credit.amount,105.0,0.105,87.0,18.0,0.171429,-0.728238,0.046815,0.114711,0.055238,0.117619,1800.0
"[1800.0, 4400.0)",credit.amount,407.0,0.407,298.0,109.0,0.267813,-0.158448,0.009884,0.114711,0.117619,0.117619,4400.0
"[4400.0, inf)",credit.amount,221.0,0.221,130.0,91.0,0.411765,0.490623,0.057707,0.114711,0.000000,0.117619,inf
special,credit.amount,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.114711,0.000000,0.117619,special
missing,credit.amount,0.0,0.000,0.0,0.0,NaN,0.000000,0.000000,0.114711,0.000000,0.117619,missing


In [30]:
bin_w['credit.amount']

,variable,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,ks,ks_max,breaks
bin,,,,,,,,,,,,
"[-inf, 1400.0)",credit.amount,1932,0.264658,1850.0,82.0,0.042443,0.033661,0.000305,0.16421,0.009048,0.146667,1400.0
"[1400.0, 1800.0)",credit.amount,888,0.121644,870.0,18.0,0.020270,-0.728238,0.046815,0.16421,0.055238,0.146667,1800.0
"[1800.0, 2000.0)",credit.amount,411,0.056301,390.0,21.0,0.051095,0.228259,0.003261,0.16421,0.040952,0.146667,2000.0
"[2000.0, 3800.0)",credit.amount,2346,0.321370,2280.0,66.0,0.028133,-0.392393,0.041482,0.16421,0.146667,0.146667,3800.0
"[3800.0, inf)",credit.amount,1723,0.236027,1610.0,113.0,0.065583,0.493281,0.072348,0.16421,0.000000,0.146667,inf
special,credit.amount,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.16421,0.000000,0.146667,special
missing,credit.amount,0,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.16421,0.000000,0.146667,missing
